In [1]:
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np

from utils import TextLoader

# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'data/tinyshakespeare' # 셰익스피어 희곡 <리처드 3세> 데이터로 학습
#data_dir = 'data/linux' # <Linux 소스코드> 데이터로 학습
batch_size = 50 # Training : 50, Sampling : 1
seq_length = 50 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수
learning_rate = 0.002
num_epochs = 2
num_hidden_layers = 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime = u' '         # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    ret = prime       # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime[-1]   # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    for n in range(num_sampling):
        x = np.zeros((1, 1))
        x[0, 0] = vocab[char]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        ret += pred     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)

reading text file


W0824 11:40:55.071004  9320 deprecation.py:323] From <ipython-input-1-3e27b468f9d8>:44: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0824 11:41:07.048928  9320 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0824 11:41:07.048928  9320 deprecation.py:323] From <ipython-input-1-3e27b468f9d8>:48: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is e

0(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.943
1(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.217
2(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 4.112
3(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.936
4(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.707
5(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.509
6(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.399
7(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.378
8(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.396
9(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.274
10(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.169
11(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.161
12(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.187
13(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.123
14(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 3.051
15(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 2.947
16(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(los

138(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.913
139(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.898
140(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.906
141(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.909
142(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.888
143(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.899
144(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.891
145(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.861
146(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.898
147(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.904
148(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.898
149(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.891
150(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.967
151(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.931
152(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.957
153(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.853
154(학습한 배치개수)/924(학습할 배치

273(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.695
274(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.706
275(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.697
276(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.722
277(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.649
278(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.678
279(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.685
280(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.712
281(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.716
282(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.754
283(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.658
284(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.685
285(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.773
286(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.699
287(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.737
288(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.680
289(학습한 배치개수)/924(학습할 배치

408(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.693
409(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.707
410(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.735
411(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.614
412(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.588
413(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.594
414(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.566
415(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.596
416(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.593
417(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.632
418(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.617
419(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.605
420(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.550
421(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.610
422(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.594
423(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 1.578
424(학습한 배치개수)/924(학습할 배치

543(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.526
544(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.513
545(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.614
546(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.527
547(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.591
548(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.605
549(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.558
550(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.536
551(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.549
552(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.592
553(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.505
554(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.485
555(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.498
556(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.497
557(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.507
558(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.554
559(학습한 배치개수)/924(학습할 배치

678(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.555
679(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.536
680(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.511
681(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.512
682(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.487
683(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.520
684(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.574
685(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.524
686(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.509
687(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.462
688(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.529
689(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.562
690(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.496
691(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.497
692(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.528
693(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.595
694(학습한 배치개수)/924(학습할 배치

813(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.504
814(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.449
815(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.525
816(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.483
817(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.413
818(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.491
819(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.538
820(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.557
821(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.557
822(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.467
823(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.531
824(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.411
825(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.396
826(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.428
827(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.441
828(학습한 배치개수)/924(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 1.508
829(학습한 배치개수)/924(학습할 배치